In [74]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

In [75]:
INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500

BATCH_SIZE = 100

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARAZTION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

In [76]:
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights2) + biases2
    else:
        layer2 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + 
                               avg_class.average(biases1))
        return tf.matmul(layer2, avg_class.average(weights2)) + avg_class.average(biases2)

In [77]:
def train(mnist):
    
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name = 'x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name = 'y-input')
    
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev = 0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape = [LAYER1_NODE]))
    
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev = 0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape = [OUTPUT_NODE]))
    
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    global_step = tf.Variable(0, trainable = False)
    
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)
    
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
       logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    regularization  = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularization 
    #上一行找了半天的bug 失之毫厘谬以千里 注意英文拼写！！！！！ 在打loss的时候打成了cross_entropy！！ 细心
    
    # 设置指数衰减的学习率。
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name = 'train')
    
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        validate_feed = {x : mnist.validation.images,
                         y_: mnist.validation.labels}
        
        test_feed = {x : mnist.test.images,
                     y_: mnist.test.labels}
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict = validate_feed)
                print("After %d step(s), validation accuracy "
                     "using average model is %g " % (i ,validate_acc))
                
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict = {x :xs, y_:ys})
            
        test_acc=sess.run(accuracy,feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" %(TRAINING_STEPS, test_acc)))
   
    

In [78]:
mnist = input_data.read_data_sets("/home/visit/learn_tf/ch5/mnist", one_hot = True)

Extracting /home/visit/learn_tf/ch5/mnist/train-images-idx3-ubyte.gz
Extracting /home/visit/learn_tf/ch5/mnist/train-labels-idx1-ubyte.gz
Extracting /home/visit/learn_tf/ch5/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/visit/learn_tf/ch5/mnist/t10k-labels-idx1-ubyte.gz


In [79]:
train(mnist)

After 0 step(s), validation accuracy using average model is 0.0956 
After 1000 step(s), validation accuracy using average model is 0.9766 
After 2000 step(s), validation accuracy using average model is 0.9816 
After 3000 step(s), validation accuracy using average model is 0.9838 
After 4000 step(s), validation accuracy using average model is 0.9838 
After 5000 step(s), validation accuracy using average model is 0.985 
After 6000 step(s), validation accuracy using average model is 0.9848 
After 7000 step(s), validation accuracy using average model is 0.986 
After 8000 step(s), validation accuracy using average model is 0.984 
After 9000 step(s), validation accuracy using average model is 0.9844 
After 10000 step(s), validation accuracy using average model is 0.9856 
After 11000 step(s), validation accuracy using average model is 0.9852 
After 12000 step(s), validation accuracy using average model is 0.985 
After 13000 step(s), validation accuracy using average model is 0.9852 
After 140